In [19]:
### --- default imports ---

import json
import os
import pickle
from itertools import chain
import pandas as pd

In [30]:
### --- constants definitions ---

DATA_BASE = "../../master_cloud/corpora"
ETL_BASE = "preprocessed"
ETL_PATH = os.path.join(DATA_BASE, ETL_BASE)

# standard meta data fields
DATASET = 'dataset'
SUBSET = 'subset'
ID = 'doc_id'
ID2 = 'doc_subid'
TITLE = 'doc_title'
CAT = 'doc_category'
META = [DATASET, SUBSET, CAT, ID, ID2, TITLE]

In [56]:
### --- extract, transform, load (save) the following corpus:

CORPUS = "OnlineParticipation"


def transform_subset(orig_data, subset: str):
    """
    :param orig_data: list of dictionaries in original key/value format
    :param subset: string identifier of the subset the data belongs to
    :return: list of dictionaries in standard key/value format
    """
    category_lookup = {}
    print('process', subset)

    meta_data = []
    text_data = []
    for d in orig_data:
        nd = dict()
        nd[ID] = d['suggestion_id']
        # print(subset, nd[ID])
        nd[DATASET] = CORPUS
        nd[SUBSET] = subset
        nd[TITLE] = d['title']

        # wuppertal has a different data scheme
        if subset == 'wuppertal2017':
            if 'tags' in d:
                nd[CAT] = tuple(d['tags'])
                category_lookup[nd[ID]] = nd[CAT]
            else:
                nd[CAT] = category_lookup[nd[ID]]
            nd[ID2] = 0
            text = d['title'] + ' .\n' \
                + d['content'] + ' .\n' \
                + d['Voraussichtliche Rolle für die Stadt Wuppertal'] + ' .\n' \
                + d['Mehrwert der Idee für Wuppertal'] + ' .\n'
                # + d['Eigene Rolle bei der Projektidee'] + ' .\n'
                # + d['Geschätzte Umsetzungsdauer und Startschuss'] + ' .\n'
                # + d['Kostenschätzung der Ideeneinreicher'] + ' .\n'

        else:
            if 'category' in d:
                nd[CAT] = d['category']
                category_lookup[nd[ID]] = nd[CAT]
            else:
                nd[CAT] = category_lookup[nd[ID]]
            nd[ID2] = d['comment_id'] if ('comment_id' in d) else 0
            # ignore if no content
            if not d['content']:
                continue
            text = d['title'] + ' .\n' + d['content'] if d['title'] else d['content']

        metahash = hash((nd[DATASET], nd[SUBSET], nd[ID], nd[ID2], nd[TITLE], nd[CAT]))
        meta = (metahash, nd)
        meta_data.append(meta)
        text_data.append((metahash, text))
        # print(metahash, ':', nd[TITLE])

    return meta_data, text_data


def participation_data(items: int=None, start: int=0):
    """
    :param items: number of items (subsets) to process in one call (None for no limit)
    :param start: index of first item (subset) to process
    :yield: data set name, data subset name, data json
    """
    
    print("process", CORPUS)
    
    # --- open files ---
    local_path = "OnlineParticipationDatasets/downloads"
    full_path = os.path.join(DATA_BASE, local_path)

    for root, dirs, files in os.walk(full_path, topdown=False):
        if items:
            items += start
            if items > len(files):
                items = None

        for name in files[start:items]:
            if name[-9:-5] == 'flat':
                fpath = os.path.join(root, name)
                try: 
                    with open(fpath, 'r') as fp:
                        print('open:', fpath)
                        data = json.load(fp)
                        if not data:
                            continue
                except IOError:
                    print("Could not open", fpath)
                    continue
                subset = name[6:-10]
                
                yield transform_subset(data, subset)


meta_data, text_data = zip(*[item for item in participation_data()])

process OnlineParticipation
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_bonn2017_flat.json
process bonn2017
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_badgodesberg_flat.json
process badgodesberg
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_bonn2019_flat.json
process bonn2019
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_bonn2011_flat.json
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_raddialog-koeln_flat.json
process raddialog-koeln
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_koeln2012_flat.json
process koeln2012
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_koeln2016_flat.json
process koeln2016
open: ../../master_cloud/corpora/OnlineParticipationDatasets/downloads/items_bonn2015_flat.json
process bonn2015
open: ../../master_cloud/corpora/OnlineParticipationDataset

In [58]:
# generate DataFrame for meta data (hash values as index)
meta_flat = list(chain(*meta_data))
meta_df = pd.DataFrame.from_items(meta_flat).T
meta_df = df[~df.index.duplicated(keep='first')]  # remove duplicates

# generate DataFrame for content data (hash values as index)
text_flat = dict(chain(*text_data))  # removes duplicates as well
text_ser = pd.Series(text_flat)

assert len(meta_df) == len(text_ser)

# store meta data and content
path = os.path.join(ETL_PATH, CORPUS)
os.makedirs(path, exist_ok=True)
meta_file = os.path.join(path, 'meta_dataframe.pickle')
text_file = os.path.join(path, 'text_series.pickle')

print('saving to', meta_file)
meta_df.to_pickle(meta_file)
print('saving to', text_file)
text_ser.to_pickle(text_file)

save ../../master_cloud/corpora/preprocessed/OnlineParticipation/meta_dataframe.pickle
save ../../master_cloud/corpora/preprocessed/OnlineParticipation/text_series.pickle


In [47]:
meta_df

,dataset,doc_category,doc_id,doc_subid,doc_title,subset
-439894440295794811,OnlineParticipation,None,985,0,Nebentätigkeiten von OB und Kommunalpoilter - ...,bonn2017
-241821531554870882,OnlineParticipation,None,988,0,Wache (Gabi) im Bahnhof,bonn2017
-3189585642850737045,OnlineParticipation,Finanzen und Beteiligung,986,0,Monetäre Sparmaßnahme,bonn2017
-3958532978963917503,OnlineParticipation,Finanzen und Beteiligung,987,0,Kosteneinsparung bei Friedhof-/Parkanlagen in ...,bonn2017
452979505005287037,OnlineParticipation,None,983,0,Leerstand,bonn2017
-1233941054549180989,OnlineParticipation,None,979,0,Minusstunden bei der Feuerwehr; Öffnungszeiten...,bonn2017
-6916208575326518865,OnlineParticipation,None,980,0,Stadtverwaltung und Einsparmöglichkeiten,bonn2017
2101409118939873410,OnlineParticipation,None,982,0,Aufsichtsrat Stadtwerke Bonn,bonn2017
8112376260580311433,OnlineParticipation,None,984,0,Sparkasse Köln Bonn und Ausschüttung der Gewinne,bonn2017
-1367476038995833645,OnlineParticipation,Freizeit und Sport,990,0,Beendigung der Unterstützung der Kunst!Rasen-V...,bonn2017


In [59]:
#pd.read_pickle(meta_file)
pd.read_pickle(text_file)

-9222762856206789549           innerGRÜN... .\nWunderbare Idee! Danke! <3
-9221398648245546881    Subventionen sind keine Geschenke .\nSubventio...
-9220330193953881393    Benutzungspflicht nicht sinnvoll .\nDie Benutz...
-9218277033136026890    Anwohnerparkausweise sind der .\nAnwohnerparka...
-9218227822569336850    Zentrum für Alte Musik in den Helioshöfen .\nI...
-9217827182021413522    RE: Wenn es was nützen würde. .\n@#6 Ich kann ...
-9217783665984115847    Berliner Flughafen .\nWenn ihr ehrenamtlich Tä...
-9217694594524019701    Verbesserung Belag .\nSehr geehrte Damen und H...
-9217310010396714308    Radweg zugeparkt .\nAuf Höhe des Kindergartens...
-9215968989273724015    Gibt wichtigeres .\nVielleicht mochte der Herr...
-9215920164235377221    Leerstände in Tannenbusch .\nHallo,der Artikel...
-9214862277269507509     Schlagloch .\nAuf der Straße ist ein Schlagloch.
-9212789436452858685    Unsoziale Vorschläge scheinen in Mode zu sein!...
-9212180652786899332    Nicht nur erhö

In [48]:
text_ser

-9222705053480570618    Mehr KOntrollen und höhere .\nMehr KOntrollen ...
-9221550976700310107    Radwege Zündorf - Langel .\nRadwege sind das E...
-9221417250383768694    B-190 .\nAbgelehnte Asylbewerber und augensche...
-9219176285779956486    Abstellanlagen .\nBei Veranstaltungen (Flohmar...
-9218273747915158131    In solchen Fällen sollten .\nIn solchen Fällen...
-9218105424201763225    Re: Sparen Verkehrsregelung durch mehr KReisve...
-9217849670039011609    Re: Sparen Verkehrsregelung durch mehr KReisve...
-9216920857899606521    Richtig, es gibt viele .\nRichtig, es gibt vie...
-9216555619309553328    Benutzungspflicht aufheben .\nBenutzungspflich...
-9215629446860450611    Re: Sparen Verkehrsregelung durch mehr KReisve...
-9215027730340189120    Keine Kürzungen in den .\nKeine Kürzungen in d...
-9214441679459175101    Radwegmarkierung .\nWer die Kölnstraße stadtei...
-9214222029094837432    Sehr gefährlich .\nAutofahren übersehen oft da...
-9214117125193208370    Anpflanzung vo

In [51]:
# show duplicates
s = dict()
for k, v in meta_flat:
    if k in s:
        print(k)
        print(v)
        print(s[k])
        print()
    else:
        s[k] = v

-5760737640325627952
{'doc_id': 3067, 'dataset': 'OnlineParticipation', 'subset': 'badgodesberg', 'doc_title': 'schön wäre mulikulti, aber eher monkulti', 'doc_category': 'Beleuchtung verbessern', 'doc_subid': 6633}
{'doc_id': 3067, 'dataset': 'OnlineParticipation', 'subset': 'badgodesberg', 'doc_title': 'schön wäre mulikulti, aber eher monkulti', 'doc_category': 'Beleuchtung verbessern', 'doc_subid': 6633}

-1985777127743097559
{'doc_id': 3008, 'dataset': 'OnlineParticipation', 'subset': 'badgodesberg', 'doc_title': 'In die "Kleinstmoscheen"', 'doc_category': 'Kulturangebote verbessern', 'doc_subid': 6494}
{'doc_id': 3008, 'dataset': 'OnlineParticipation', 'subset': 'badgodesberg', 'doc_title': 'In die "Kleinstmoscheen"', 'doc_category': 'Kulturangebote verbessern', 'doc_subid': 6494}

7654827991124309798
{'doc_id': 2959, 'dataset': 'OnlineParticipation', 'subset': 'badgodesberg', 'doc_title': 'legendäre Sätze', 'doc_category': 'Sonstiges', 'doc_subid': 6314}
{'doc_id': 2959, 'dataset